In [3]:
import pandas as pd

In [4]:
test = pd.Series({'a':1, 'b':2, 'c':3})
test.a

1

# Using finetuned model

Created the fine tuned model for version 1 in prep_dataset.ipynb

This is to check if it works, and see how well it works

In [ ]:
from transformers import AutoTokenizer, GenerationConfig, LlamaForCausalLM, LlamaTokenizer
import torch
from peft import PeftModel

base_model = "meta-llama/Llama-2-7b-hf"

load_8bit: bool = False
base_model: str = ""
lora_weights: str = "tloen/alpaca-lora-7b"
prompt_template: str = ""  # The prompt template to use, will default to alpaca.
server_name: str = "0.0.0.0"  # Allows to listen on all interfaces by providing '0.
share_gradio: bool = False

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:

model = LlamaForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=load_8bit,
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(
    model,
    "./results",
    torch_dtype=torch.float16,
)

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    "./results",
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True
)

In [ ]:
import torch
from peft import PeftModel    
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer

model_name = "meta-llama/Llama-2-7b-hf"
adapters_name = "/home/msaad/workspace/honors-thesis/fine-tuning/results/final_checkpoint"

print(f"Starting to load the model {model_name} into memory")

m = AutoModelForCausalLM.from_pretrained(
    model_name,
    #load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    device_map={"": 0}
)
m = PeftModel.from_pretrained(m, adapters_name)
m = m.merge_and_unload()
tok = LlamaTokenizer.from_pretrained(model_name)
tok.bos_token_id = 1

stop_token_ids = [0]

print(f"Successfully loaded the model {model_name} into memory")

In [ ]:
config = LoraConfig.from_pretrained("timdettmers/guanaco-65b")
config.inference_mode = False 

model = get_peft_model(model, config)
print_trainable_parameters(model)

In [ ]:
text = "What school is this?"
inputs = tok(text, return_tensors="pt").to("cuda")
outputs = m.generate(input_ids=inputs["input_ids"].to("cuda"), attention_mask=inputs["attention_mask"], max_new_tokens=50, pad_token_id=tok.eos_token_id)

print(tok.decode(outputs[0], skip_special_tokens=True))

In [ ]:
outputs = m.generate(
    input_ids=inputs["input_ids"].to("cuda"), 
    attention_mask=inputs["attention_mask"], 
    max_new_tokens=50, 
    pad_token_id=tok.eos_token_id, 
    #do_sample=True, 
    top_k=50
)
print(tok.decode(outputs[0], skip_special_tokens=True))

# New test

In [ ]:
import torch
from peft import PeftModel    
from transformers import AutoModelForCausalLM, LlamaTokenizer
from peft import LoraConfig, get_peft_model

model_name = "meta-llama/Llama-2-7b-hf"
adapters_name = "/home/msaad/workspace/honors-thesis/fine-tuning/results/final_checkpoint"

print(f"Starting to load the model {model_name} into memory")

m = AutoModelForCausalLM.from_pretrained(
    model_name,
    #load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    device_map={"": 0}
)
config = LoraConfig.from_pretrained(adapters_name)
config.inference_mode = True 

m = get_peft_model(m, config)

tok = LlamaTokenizer.from_pretrained(model_name)
tok.bos_token_id = 1

stop_token_ids = [0]

print(f"Successfully loaded the model {model_name} into memory")

In [ ]:
text = "What are steps to apply?"
inputs = tok(text, return_tensors="pt").to("cuda")
outputs = m.generate(input_ids=inputs["input_ids"].to("cuda"), attention_mask=inputs["attention_mask"], max_new_tokens=50, pad_token_id=tok.eos_token_id)

print(tok.decode(outputs[0], skip_special_tokens=True))

In [ ]:
import torch
from peft import PeftModel    
from transformers import AutoModelForCausalLM, LlamaTokenizer
from peft import LoraConfig, get_peft_model

model_name = "meta-llama/Llama-2-7b-hf"
adapters_name = "/home/msaad/workspace/honors-thesis/fine-tuning/results/final_checkpoint"

print(f"Starting to load the model {model_name} into memory")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    #load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    device_map={"": 0}
)

model = PeftModel.from_pretrained(model_name, adapters_name, is_trainable=True)